In [ ]:
backtest_start = '2024-08-01'
backtest_end = '2025-07-31'

# Subset returns
returns_bt = returns.loc[backtest_start:backtest_end]

In [ ]:
def simulate_portfolio(returns, weights, rebalance_freq='M'):
    # Resample dates for rebalancing
    dates = returns.resample(rebalance_freq).first().index
    portfolio_return = []

    weights = np.array(list(weights.values()))
    prev_weights = weights.copy()

    for i in range(len(returns)):
        date = returns.index[i]
        row_returns = returns.iloc[i].values
        
        # Portfolio return
        port_ret = np.sum(prev_weights * row_returns)
        portfolio_return.append(port_ret)

        # Rebalance at month end
        if date in dates:
            total_growth = np.sum(prev_weights * (1 + row_returns))
            new_weights = (prev_weights * (1 + row_returns)) / total_growth
            prev_weights = weights  # Reset to target weights

    return np.array(portfolio_return)

# Our Strategy (Max Sharpe Weights)
strategy_weights = {'TSLA': 0.32, 'BND': 0.48, 'SPY': 0.20}
strat_returns = simulate_portfolio(returns_bt, strategy_weights)

# Benchmark: 60% SPY / 40% BND
benchmark_weights = {'TSLA': 0.0, 'BND': 0.40, 'SPY': 0.60}
bench_returns = simulate_portfolio(returns_bt, benchmark_weights)

In [ ]:
import pandas as pd

cum_strat = (1 + pd.Series(strat_returns)).cumprod()
cum_bench = (1 + pd.Series(bench_returns)).cumprod()

plt.figure(figsize=(14, 7))
plt.plot(cum_strat.index, cum_strat * 10000, label='Optimized Portfolio (Max Sharpe)', linewidth=2)
plt.plot(cum_bench.index, cum_bench * 10000, label='60/40 SPY-BND Benchmark', linestyle='--', linewidth=2)

plt.title('Backtest: Optimized Portfolio vs 60/40 Benchmark (Aug 2024 – Jul 2025)')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
def performance_metrics(returns, name):
    total_return = (np.prod(1 + returns) - 1) * 100
    ann_return = np.mean(returns) * 252 * 100
    ann_vol = np.std(returns) * np.sqrt(252) * 100
    sharpe = (np.mean(returns) * 252 - 0.03) / (np.std(returns) * np.sqrt(252))
    print(f"{name}")
    print(f"  Total Return: {total_return:.2f}%")
    print(f"  Annualized Return: {ann_return:.2f}%")
    print(f"  Annualized Volatility: {ann_vol:.2f}%")
    print(f"  Sharpe Ratio (rf=3%): {sharpe:.3f}")
    print("  ---")
    return {
        'Total Return (%)': total_return,
        'Annual Return (%)': ann_return,
        'Volatility (%)': ann_vol,
        'Sharpe Ratio': sharpe
    }

metrics = []
metrics.append(performance_metrics(strat_returns, "Optimized Portfolio"))
metrics.append(performance_metrics(bench_returns, "60/40 Benchmark"))